In [1]:
from datetime import datetime
from decimal import Decimal

import numpy as np
import talib as ta

from btopt.data.bar import Bar
from btopt.data.dataloader import CSVDataLoader
from btopt.engine import Engine
from btopt.log_config import logger_main
from btopt.order import Order
from btopt.strategy.strategy import Strategy

In [2]:
class Empty(Strategy):
    def __init__(
        self,
    ) -> None:
        super().__init__()

    def on_data(self, bar: Bar) -> None:
        x = Engine()
        x._dataview.data

        if len(self) <= 1:
            return

        self.debug(f"PRIMARY: {(self.data.close[1])}, TIMEFRAME: {self.datas.keys()}")
        self.debug(f"TIMEFRAMES: {self.datas[self._primary_symbol].timeframes}")
        # self.debug(
        #     f"SECONDARY: {(self.datas[self._primary_symbol].get(value="close"))}, TIMEFRAME: {self.datas[self._primary_symbol][1]._timeframe}"
        # )

        # if len(self) > 100:
        #     raise

    @staticmethod
    def debug(message):
        logger_main.info(message)


class SimpleMovingAverageCrossover(Strategy):
    def __init__(self, fast_period: int = 10, slow_period: int = 20):
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.warmup_period = 1

        self.fast_ma = None
        self.slow_ma = None

        self.total = 0

    def on_data(self) -> None:
        bar = self.datas[self._primary_symbol].get(index=0)
        close_prices = self.datas[self._primary_symbol].get(
            size=self.slow_period, value="close"
        )[::-1]

        if len(close_prices) < self.slow_period:
            return

        fast_ma = ta.SMA(
            np.array(close_prices).astype(float)[::-1], timeperiod=self.fast_period
        )[-1]
        slow_ma = ta.SMA(
            np.array(close_prices).astype(float)[::-1], timeperiod=self.slow_period
        )[-1]

        formatted_time = datetime.fromtimestamp(bar.timestamp / 1_000_000_000).strftime(
            "%Y-%m-%d, %H:%M:%S"
        )
        self.log(
            f"\n–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––\nDate: {formatted_time} | Close: {bar.close} | \nPrevious: {[float(x) for x in close_prices]}"
        )
        # # current_position = self.get_current_position(bar.ticker)
        # if (self.fast_ma is not None) and (self.slow_ma is not None):
        #     prev_diff = self.fast_ma - self.slow_ma
        #     diff = fast_ma - slow_ma

        #     if (diff > 0) and (prev_diff <= 0):
        #         self.log("Bullish crossover.")
        #         # # Bullish crossover
        #         # if current_position <= 0:
        #         #     size = abs(current_position) + 1
        #         #     self.buy(bar.ticker, size)
        #         #     self.total += 1
        #     elif (diff < 0) and (prev_diff >= 0):
        #         self.log("Bearish crossover.")
        #         # # Bearish crossover
        #         # if current_position >= 0:
        #         #     size = current_position + 1
        #         #     self.sell(bar.ticker, size)
        #         #     self.total += 1

        # Update MA values for next iteration
        self.fast_ma = fast_ma
        self.slow_ma = slow_ma

    def on_order_update(self, order: Order) -> None:
        logger_main.info(f"Order update: {order}")

    def on_trade_update(self, trade) -> None:
        logger_main.info(f"Trade update: {trade}")

In [3]:
def run_backtest():
    # Initialize the engine
    engine = Engine()

    # Load data
    symbol = "EURUSD"
    start_date = "2022-01-01"
    end_date = "2023-01-01"

    ctf, htf = "1h", "1d"

    dataloader = CSVDataLoader(symbol, "1m", start_date=start_date, end_date=end_date)

    # engine.resample_data(dataloader, ctf)
    engine.resample_data(dataloader, htf)

    # # Create and add the strategy
    # strategy = SimpleMovingAverageCrossover(
    #     "SMA Crossover", fast_period=10, slow_period=20
    # )

    # engine.add_strategy(Empty, ctf, htf)
    engine.add_strategy(SimpleMovingAverageCrossover, fast_period=10, slow_period=20)

    # Set up the backtest configuration

    initial_capital = Decimal("100000")
    commission_rate = Decimal("0.001")  # 0.1% commission
    config = {
        "initial_capital": initial_capital,
        "commission_rate": commission_rate,
    }
    engine.set_config(config)

    # Run the backtest
    try:
        logger_main.info("Starting backtest")
        reporter = engine.run()

        return reporter
    except Exception as e:
        logger_main.error(f"Error during backtest: {e}", exc_info=True)

In [4]:
reporter = run_backtest()

Log file '/Users/jerryinyang/Code/btopt/logs/main.log' has been cleared successfully.


In [5]:
reporter.calculate_calmar_ratio()

AttributeError: 'NoneType' object has no attribute 'calculate_calmar_ratio'

In [ ]:
if reporter:
    print("Backtest completed successfully.")
    print("Performance Summary:")
    print(reporter.generate_performance_summary())
else:
    print("Backtest failed.")

In [ ]:
reporter